In [1]:
# Library imports
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from helper import print_results, metis_map

# Prunning Youtube Network for Analysis

In [ ]:
youtubeG = nx.read_edgelist("data/com-youtube.ungraph.txt", create_using = nx.Graph, nodetype = int)

The next cell will make the youtube network much smaller so we can 

In [ ]:

# Obtains list of degrees that have degree less than 3
degree_Thres = 6
remove = [node for node,degree in dict(youtubeG.degree()).items() if  degree < degree_Thres]

# Removes nodes with small degree and creates sparse matrix
youtubeG.remove_nodes_from(remove)
nx.write_edgelist(youtubeG,'youtube_sparse.txt', data=False)


# Cluster Analysis

### Data Imports
Load in every graph. We'll look at some simple metrics to get some baseline understanding.

In [12]:
wikiG = nx.read_edgelist("input/wiki-Vote.txt", create_using = nx.Graph, nodetype = int)
collabG = nx.read_edgelist("input/ca-GrQc.txt", create_using = nx.Graph, nodetype = int)
p2pG = nx.read_edgelist("input/p2p-Gnutella08.txt", create_using = nx.Graph, nodetype = int)
fbG = nx.read_edgelist("input/facebook_combined.txt", create_using = nx.Graph, nodetype = int)
youtubeG = nx.read_edgelist("input/com-youtube.ungraph.txt", create_using = nx.Graph, nodetype = int)
    
pd.DataFrame({"Graph": ["Wikipedia", "Collaborations", "Gnutella", "Facebook", "Youtube"],
    "Node Count": [wikiG.number_of_nodes(), collabG.number_of_nodes(), p2pG.number_of_nodes(), fbG.number_of_nodes(), youtubeG.number_of_nodes()],
    "Edge Count": [wikiG.number_of_edges(), collabG.number_of_edges(), p2pG.number_of_edges(), fbG.number_of_edges(), youtubeG.number_of_edges()]})

,Graph,Node Count,Edge Count
0,Wikipedia,7115,100762
1,Collaborations,5242,14496
2,Gnutella,6301,20777
3,Facebook,4039,88234
4,Youtube,1134890,2987624


We use the following code to find a mapping between our Node_IDs from our original text files to our Node_IDs in the METIS files. Note that all helper functions can be found in helper.py.

In [ ]:
names = ["wiki-Vote", "p2p-Gnutella08", "facebook_combined", "ca-GrQc", "com-youtube.ungraph"]
for i in names:
    fname = "input/" + i + ".txt"
    obj_fname = "MetisAlgo/" + i + "_map.obj"
    metis_map(fname, obj_fname)

### Wiki-Vote

Now we'll pull the three metrics (modularity, ncut, and condunctance) for clusterings without ground truth. We can get a good idea for how well each algorithm was able to group nodes with high edge density together.

In [4]:
name = names[0]
mcl = print_results(wikiG, name, "mcl")
metis = print_results(wikiG, name, "metis")
community = print_results(wikiG, name, "community")

According to modularity, it seems as though the Clauset-Newman-Moore algorithm performed the best. As we can recall from Lab 1, the Wikipedia Vote network was our only directed graph. Unfortunately, none of our clustering algorithms were compatible with this information, so we had to treat it as undirected. The low modularity values for MCL and METIS could be attributed to this fact.



In [7]:
print("Modularity of MCL:", mcl.Modularity[0])
print("Modularity of METIS:", metis.Modularity[0])
print("Modularity of Communities:", community.Modularity[0])

Modularity of MCL: 0.01080145681798467
Modularity of METIS: 0.004638421479492384
Modularity of Communities: 0.35004242922498335


### Peer-To-Peer Filesharing

In [ ]:
name = names[1]
mcl = print_results(wikiG, name, "mcl")
metis = print_results(wikiG, name, "metis")
community = print_results(wikiG, name, "community")

### Facebook

In [ ]:
name = names[0]
mcl = print_results(wikiG, name, "mcl")
metis = print_results(wikiG, name, "metis")
community = print_results(wikiG, name, "community")

### Collaborations

In [18]:
collabG.remove_nodes_from([12295])
print_results(collabG, "ca-GrQc", "mcl")

,Modularity,n-cut,Conductance
0,0.001119,1.000000,1.000000
1,0.001119,1.000000,1.000000
2,0.001119,1.000000,1.000000
3,0.001119,0.991189,0.991189
4,0.001119,1.000000,1.000000
...,...,...,...
748,0.001119,1.000000,1.000000
749,0.001119,1.000000,1.000000
750,0.001119,1.000000,1.000000
751,0.001119,1.000000,1.000000


In [11]:
collab = np.loadtxt("input/ca-GrQc.txt", dtype = int)
for i in collab:
    if i[0] == i[1]:
        print(i)

[16703 16703]
[11372 11372]
[1343 1343]
[4442 4442]
[18314 18314]
[11318 11318]
[25777 25777]
[14840 14840]
[6648 6648]
[13 13]
[4685 4685]
[12295 12295]


In [10]:
for c in nx.connected_components(collabG):
    if len(c) == 1:
        print(c)

{12295}


### Youtube

In [ ]:
print_results(youtubeG, names[4], "mcl")